In [1]:
from process import preprocess
import plotly.offline
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import pandas as pd

In [2]:
from pathlib import Path
data_path = Path('../data/NILU_Dataset_Trondheim_2014-2019.csv')

In [3]:
    df = pd.read_csv(data_path, index_col=[0], header=[0, 1])
    
    df.index.name = 'timestamp'
    df.index = pd.to_datetime(df.index)

In [4]:
len(df)

45158

In [5]:
config = {
    'pred_var': 'Torvet PM10', # Must include station and pollutants name (column name)
    'stations': ['Torvet'], # Stations to use in feature extraction
    'window': 6,
    'load': True,
    'freeze1': True,
    'freeze2': True,
    'freeze3': True,
    'freeze4': True,
    'load_state': True,
    'state': 6, #Train estimate of state* hour ahead, 6,12,24,36,48
    'core_features': 5
}

data = preprocess(**config)

In [6]:
config['window']

6

In [7]:
import Deep_meta_learner as MLP
MLP.train(config, data)


RuntimeError: Error(s) in loading state_dict for Core:
	size mismatch for FC_6_hour.weight: copying a param with shape torch.Size([48, 640]) from checkpoint, the shape in current model is torch.Size([6, 640]).
	size mismatch for FC_6_hour.bias: copying a param with shape torch.Size([48]) from checkpoint, the shape in current model is torch.Size([6]).
	size mismatch for FC_12_hour.weight: copying a param with shape torch.Size([48, 640]) from checkpoint, the shape in current model is torch.Size([6, 640]).
	size mismatch for FC_12_hour.bias: copying a param with shape torch.Size([48]) from checkpoint, the shape in current model is torch.Size([6]).
	size mismatch for FC_24_hour.weight: copying a param with shape torch.Size([48, 640]) from checkpoint, the shape in current model is torch.Size([6, 640]).
	size mismatch for FC_24_hour.bias: copying a param with shape torch.Size([48]) from checkpoint, the shape in current model is torch.Size([6]).
	size mismatch for FC_36_hour.weight: copying a

In [ ]:
print('X train', data['X_train'].shape)
print('y train', data['y_train'].shape)
print('X validation', data['X_val'].shape)
print('X test', data['X_test'].shape)



**Train Multi Output RF | GBM | MLP**

_Params are hidden inside each file_

In [32]:
import SMLP as SMLP
SMLP.train(config, data)

In [25]:
smlp_results, rmse, r2 = SMLP.predict(config, data)
print('RMSE: {:.2f}   R2:{:.2f}'.format(rmse, r2))

RMSE: 95.27   R2:-40.67


In [10]:
import GBM as GBM
GBM.train(config, data)
gbm_results, rmse, r2 = GBM.predict(config, data)
print('RMSE: {:.2f}   R2:{:.2f}'.format(rmse, r2))

RMSE: 11.45   R2:0.11


In [30]:
import RF as RF
RF.train(config, data)

In [31]:
import RF as RF
RF.train(config, data)
rf_results, rmse, r2 = RF.predict(config, data)
print('RMSE: {:.2f}   R2:{:.2f}'.format(rmse, r2))

RMSE: 13.53   R2:0.16


In [8]:
mlp_results, rmse, r2 = MLP.predict(config, data)
print('RMSE: {:.2f}   R2:{:.2f}'.format(rmse, r2))

RMSE: 10.67   R2:0.10


**Plotting the results**

In [12]:
import plotly.plotly as py
import plotly.graph_objs as go
import cloudpickle
from plotly.offline import iplot, init_notebook_mode
import cufflinks
plotly.tools.set_credentials_file(username='johap', api_key='k6Uj8QR961xiIrJLYYs9')

In [13]:
MLP1 = "MLP_min feature" 
MLP1 = "MLP_min feature" 
MLP1 = "MLP_min feature" 

In [14]:
#gbm_results['RF'] = rf_results['RF']
gbm_results['MLP'] = mlp_results['MLP']
#gbm_results['SMLP'] = smlp_results['MLP']
gbm_results['2019-02-15':].iplot()